# This notebook serves as walkthrough for planning an experiment for creation through the OT2.
### The following modules are used and should be in the same directory as this notebook: 
* **CreateSamples** is responsible for calculating sample information, which includes component weight fractions and stock volumes
* **OT2Commands** is responsible for setting up information to be interpretted and executed by opentrons **BASED ON THE LATEST API 2.3 and above**
* **OT2Graphing** contains graphing tools to help visualize and explore parameter spaces.

In [7]:
import os
import warnings
os.getcwd()
ITERATION = 1
warnings.warn('This protocol is running iteration %d ...'%ITERATION)

<ipython-input-7-d70c23759152>:5: UserWarning: This protocol is running iteration 1 ...
  warnings.warn('This protocol is running iteration %d ...'%ITERATION)


In [8]:
from Plan import CreateSamples
from Prepare import OT2Commands as ALH
from opentrons import simulate, execute, protocol_api
import importlib
import pandas as pd

_**Note**_  : If your file is not in the same folder, you can specify its path by using '\' and the folder names. 

* to go up a folder use this : '../'
* to use the same folder: './' 

Be consistent with the slashes you use , either all: '\' or '/' 

In [9]:
path = r"New_Testing_Protocol_Generalized.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples.get_experiment_plan(path, chem_path)

In [10]:
stock_volumes = CreateSamples.concentration_from_csv('Volumes/Itr%d.csv'%ITERATION)
stock_volumes = stock_volumes.loc[:, ~stock_volumes.columns.str.contains('^Unnamed')]
stock_volumes['CTAB-stock'].sum(axis=0)
stock_volumes.astype(int)

,CTAB-stock,water-stock,GC-stock,AG-stock,AA-stock,SEEDS-stock
0,112,108,69,21,12,28
1,112,79,69,34,28,28
2,112,81,69,19,41,28
3,112,91,69,24,26,28
4,112,91,69,28,22,28
5,112,81,69,34,26,28
6,112,104,69,22,15,28
7,112,123,69,18,0,28
8,112,91,69,24,26,28
9,112,92,69,28,21,28


In [11]:
stock_volumes.sum(axis=0)

CTAB-stock     4592.0
water-stock    3803.0
GC-stock       2829.0
AG-stock       1083.0
AA-stock        894.0
SEEDS-stock    1148.0
dtype: float64

## Simulate

Run the following cells to simulate the robot protocol. 
If you look at the last cell in this section, you can see the actual steps the robot will follow to make the samples.

In [14]:
labware_dir_path = r"Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)
protocol = simulate.get_protocol_api('2.7', extra_labware= custom_labware_dict) #

/Users/Huat/.opentrons/robot_settings.json not found. Loading defaults
/Users/Huat/.opentrons/deck_calibration.json not found. Loading defaults


In [15]:
loaded_dict = ALH.loading_labware(protocol, plan)
max_source_vol = 17050
stock_position_info = ALH.stock_well_ranges(stock_volumes, loaded_dict, max_source_vol) 

# let's print the stock position info and make sure everything is in the right postion. 
# You can also double-check that you have enought stock to make all your samples
# (i.e. only a single range of wells is provided for each stock)

In [16]:
stock_position_info

{'CTAB-stock': {'Ranges': [[0, 41]],
  'Stock Wells': [A1 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'water-stock': {'Ranges': [[0, 41]],
  'Stock Wells': [A2 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'GC-stock': {'Ranges': [[0, 41]],
  'Stock Wells': [A3 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'AG-stock': {'Ranges': [[0, 41]],
  'Stock Wells': [A4 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'AA-stock': {'Ranges': [[0, 41]],
  'Stock Wells': [B1 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'SEEDS-stock': {'Ranges': [[0, 41]],
  'Stock Wells': [B2 of 20mLscintillation 12 Well Plate 18000 µL on 1]}}

In [17]:
Start_pos = 0

In [18]:
directions = ALH.create_sample_making_directions(stock_volumes, stock_position_info, loaded_dict, start_position=Start_pos)
ALH.pipette_volumes_component_wise(protocol, directions, loaded_dict)
# The above command will fill the wells component wise- this means that all the first stock will be dispendes in the wells that require it. 
# If you want to fill the wells individually use the following line instead:
# ALH.pipette_volumes_sample_wise(protocol,directions, loaded_dict)

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 7
Aspirating 112.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 112.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 112.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 112.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 112.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 112.0 uL into A3 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 112.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 112.0 uL into A4 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 112.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 112.0 uL into A5 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 112.0 uL from A1 of 20mLscintillat

## Execute


Run the cells below to actually execute the protocol using the robot.

Make sure that all stock solution are **not capped** and that the labware is in the correct position. 

In [12]:
# Executing 
protocol = execute.get_protocol_api('2.7', extra_labware=custom_labware_dict)

Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.


In [13]:
loaded_dict = ALH.loading_labware(protocol, plan)
stock_position_info = ALH.stock_well_ranges(stock_volumes, loaded_dict, max_source_vol) 

# let's print the stock position info and make sure everything is in the right postion. 
# You can also double-check that you have enought stock to make all your samples
# (i.e. only a single range of wells is provided for each stock)

stock_position_info

Out of bounds move: X=(418.00000000000006 motor controller, 415.3427397712944 deck) too high for limit 418.0
Out of bounds move: X=(418.00000000000006 motor controller, 415.3427397712944 deck) too high for limit 418.0


{'CTAB-stock': {'Ranges': [[0, 96]],
  'Stock Wells': [A1 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'Water-stock': {'Ranges': [[0, 96]],
  'Stock Wells': [A2 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'GC-stock': {'Ranges': [[0, 96]],
  'Stock Wells': [A3 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'AG-stock': {'Ranges': [[0, 96]],
  'Stock Wells': [A4 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'AA-stock': {'Ranges': [[0, 96]],
  'Stock Wells': [B1 of 20mLscintillation 12 Well Plate 18000 µL on 1]},
 'Seeds-stock': {'Ranges': [[0, 96]],
  'Stock Wells': [B2 of 20mLscintillation 12 Well Plate 18000 µL on 1]}}

In [14]:
# Let's generate a dictionary containing the directions to make each sample. 
# the following line generate a dictionary where the keys corresponds to the well number ( or the index fo the dataframe of sample composition). 
# The value of each key represents the components' make up of that specific sample. 
directions = ALH.create_sample_making_directions(stock_volumes, stock_position_info, loaded_dict, start_position=Start_pos)
directions

{0: {'CTAB-stock': {'Stock Position': A1 of 20mLscintillation 12 Well Plate 18000 µL on 1,
   'Destination Well Position': A1 of Corning 96 Well Plate 360 µL Flat on 6,
   'Stock Volume': 112.0},
  'Water-stock': {'Stock Position': A2 of 20mLscintillation 12 Well Plate 18000 µL on 1,
   'Destination Well Position': A1 of Corning 96 Well Plate 360 µL Flat on 6,
   'Stock Volume': 141.0},
  'GC-stock': {'Stock Position': A3 of 20mLscintillation 12 Well Plate 18000 µL on 1,
   'Destination Well Position': A1 of Corning 96 Well Plate 360 µL Flat on 6,
   'Stock Volume': 69.0},
  'AG-stock': {'Stock Position': A4 of 20mLscintillation 12 Well Plate 18000 µL on 1,
   'Destination Well Position': A1 of Corning 96 Well Plate 360 µL Flat on 6,
   'Stock Volume': 0.0},
  'AA-stock': {'Stock Position': B1 of 20mLscintillation 12 Well Plate 18000 µL on 1,
   'Destination Well Position': A1 of Corning 96 Well Plate 360 µL Flat on 6,
   'Stock Volume': 0.0},
  'Seeds-stock': {'Stock Position': B2 of 

### the following command will actully start the experiment. Once again make sure that everything is where it is supposed to be!

In [15]:
ALH.pipette_volumes_component_wise(protocol, directions, loaded_dict)

Out of bounds move: X=(418.00000000000006 motor controller, 415.3427397712944 deck) too high for limit 418.0
Out of bounds move: X=(418.00000000000006 motor controller, 415.3427397712944 deck) too high for limit 418.0
Out of bounds move: X=(418.00000000000006 motor controller, 415.3427397712944 deck) too high for limit 418.0


Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 7
Aspirating 112.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 112.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 112.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 112.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 112.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 112.0 uL into A3 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 112.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 112.0 uL into A4 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 112.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 112.0 uL into A5 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Aspirating 112.0 uL from A1 of 20mLscintillat

Aspirating 27.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 27.0 uL into B7 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at B7 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 34.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 34.0 uL into B8 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at B8 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 41.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 41.0 uL into B9 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at B9 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 7.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 7.0 uL into B11 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at B11 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 14.0 uL from A4 of 20mLscintillation 12 W

Aspirating 28.0 uL from B2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 28.0 uL into B1 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at B1 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 28.0 uL from B2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 28.0 uL into B2 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at B2 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 28.0 uL from B2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 28.0 uL into B3 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at B3 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 28.0 uL from B2 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 100.0 uL/sec
Dispensing 28.0 uL into B4 of Corning 96 Well Plate 360 µL Flat on 6 at 700.0 uL/sec
Blowing out at B4 of Corning 96 Well Plate 360 µL Flat on 6
Aspirating 28.0 uL from B2 of 20mLscintillation 12 W